In [1]:
"""
To Run the Error Labeling Pipeline


"""

'\nTo Run the Error Labeling Pipeline\n\n\n'

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2020-11-29 03:56:35,823 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-29 03:56:35,825 - settings - Setting database.user to celiib
INFO - 2020-11-29 03:56:35,826 - settings - Setting database.password to newceliipass
INFO - 2020-11-29 03:56:35,830 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-29 03:56:35,831 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-29 03:56:35,842 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-29 03:56:36,059 - settings - Setting enable_python_native_blobs to True


# configuring the virtual module

In [3]:
test_mode = True

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 400)
print(f"Sleeping {random_sleep_sec} sec before conneting")

if not test_mode:
    time.sleep(random_sleep_sec)

print("Done sleeping")
import datajoint_utils as du
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
success_flag = False
for i in range(10):
    try:
        minnie,schema = du.configure_minnie_vm()
        
    except:
        print("Locked out trying agin in 30 seconds")
        time.sleep(30)
    else:
        success_flag = True
        
        
    if success_flag:
        print("successfully configured minnie")
        break

INFO - 2020-11-29 03:56:36,090 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-29 03:56:36,091 - settings - Setting database.user to celiib
INFO - 2020-11-29 03:56:36,092 - settings - Setting database.password to newceliipass
INFO - 2020-11-29 03:56:36,094 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-29 03:56:36,289 - settings - Setting enable_python_native_blobs to True


Sleeping 276 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
successfully configured minnie


# Defining Our Table

In [5]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [6]:
#so that it will have the adapter defined
from datajoint_utils import *

In [26]:
import error_detection as ed
ed = reload(ed)

In [27]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class AutoProofreadLabels(dj.Computed):
    definition="""
    -> minnie.Decomposition
    ---
    n_face_errors : int #the number of faces that were errored out
    face_idx_for_error : longblob #the face indices for the errors computed
    n_synapses: smallint unsigned #total number of synpases
    n_errored_synapses: smallint unsigned #the number of synapses
    """
    
    
    
    key_source = (minnie.Decomposition() & "n_somas = 1" & "n_faces>500000") & "segment_id=864691134884772090"
                  
    def make(self,key):
        global_start = time.time()
        segment_id = key["segment_id"]
        verbose = True
        
        print(f"\n\n----- Working on {segment_id}-------")
        whole_pass_time = time.time()
        
        #new method that checks if the information exists in the error table and if not then 
        error_table = (minnie.DecompositionErrorLabels() & dict(segment_id=segment_id))
        if len(error_table)>0:
            print("using quick fetch")
            current_mesh = du.fetch_segment_id_mesh(segment_id)
            returned_error_faces = error_table.fetch1("face_idx_for_error")
            
        else:
            neuron_obj = (minnie.Decomposition() & key).fetch1("decomposition")

            returned_error_faces = ed.error_faces_by_axons(neuron_obj,verbose=True,visualize_errors_at_end=False)
            current_mesh = neuron_obj.mesh
            
        #------- Doing the synapse Exclusion Writing ---------- #
        data_to_write_new,n_synapses,n_errored_synapses = ed.get_error_synapse_inserts(current_mesh,
                                                                                       segment_id,
                                                                                       returned_error_faces,minnie=minnie,
                                                         return_synapse_stats=True,
                                                         verbose=True)
        
        if len(data_to_write_new)>0:
            print("Preparing to write errored synapses")
            minnie.SynapseExclude.insert(data_to_write_new,skip_duplicates=True)
            
        #------- Doing the Label Writing ---------- #
        new_key = dict(key,
                       n_face_errors = len(returned_error_faces),
                       face_idx_for_error = returned_error_faces,
                        n_synapses=n_synapses,
                        n_errored_synapses=n_errored_synapses)
        
        
        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)
        
        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
        

In [28]:
#(schema.jobs & "table_name='__decomposition_error_labels'").delete()
#minnie.SynapseExclude.delete()
#minnie.DecompositionErrorLabels.delete()

In [29]:
import time
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 900))
print('Populate Started')
if test_mode:
    AutoProofreadLabels.populate(reserve_jobs=True, suppress_errors=False)
else:
    AutoProofreadLabels.populate(reserve_jobs=True, suppress_errors=True)
print('Populate Done')

print(f"Total time for DecompositionErrorLabels populate = {time.time() - start_time}")

Populate Started


----- Working on 864691134884772090-------
Decompressing Neuron in minimal output mode...please wait
Returning empty list because there were no error faces


 ------ Total time for 864691134884772090 = 19.974971294403076 ------
Populate Done
Total time for DecompositionErrorLabels populate = 20.105098247528076


In [12]:
minnie.Decomposition() & "segment_id=864691134884772090"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),decomposition,n_vertices number of vertices,n_faces number of faces,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134884772090,0,0.25,=BLOB=,479119,973172,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,500.7524


In [13]:
global_start = time.time()
segment_id = 864691134884772090
key = dict(segment_id=segment_id,decimation_version=0,decimation_ratio=0.25)
verbose = True

print(f"\n\n----- Working on {segment_id}-------")
whole_pass_time = time.time()

#new method that checks if the information exists in the error table and if not then 
error_table = (minnie.DecompositionErrorLabels() & dict(segment_id=segment_id))
if len(error_table)>0:
    print("using quick fetch")
    current_mesh = du.fetch_segment_id_mesh(segment_id)
    returned_error_faces = error_table.fetch1("face_idx_for_error")

else:
    neuron_obj = (minnie.Decomposition() & key).fetch1("decomposition")

    returned_error_faces = ed.error_faces_by_axons(neuron_obj,verbose=True,visualize_errors_at_end=False)
    current_mesh = neuron_obj.mesh

#------- Doing the synapse Exclusion Writing ---------- #
data_to_write_new,n_synapses,n_errored_synapses = ed.get_error_synapse_inserts(current_mesh,
                                                                               segment_id,
                                                                               returned_error_faces,minnie=minnie,
                                                 return_synapse_stats=True,
                                                 verbose=True)

if len(data_to_write_new)>0:
    print("Preparing to write errored synapses")
    minnie.SynapseExclude.insert(data_to_write_new,skip_duplicates=True)



----- Working on 864691134884772090-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f14f10cb950>, <function median_mesh_center at 0x7f14f10cbbf8>, <function n_spines at 0x7f14f10cba60>, <function n_faces_branch at 0x7f14f10cb7b8>, <function skeleton_distance_branch at 0x7f14f10cb9d8>, <function spines_per_skeletal_length at 0x7f14f10cbea0>, <function no_spine_median_mesh_center at 0x7f14f10cbd08>]


ValueError: Wrong number of items passed 0, placement implies 1

In [17]:
neuron_obj.get_limb_node_names()

[]

In [19]:
import neuron_visualizations as nviz
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all",
                    )


 Working on visualization type: mesh

 Working on visualization type: skeleton
No skeletons to stack so returning empty list
Edge coordinates in plot_ipv_skeleton were of 0 length so returning
Working on  new stand alone scatter points


In [22]:
neuron_obj.n_limbs

0